In [ ]:
from research_lib.utils.data_access_utils import RDSAccessUtils
from weight_estimation.body_parts import core_body_parts
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

In [ ]:
from research_lib.utils.data_access_utils import RDSAccessUtils
from weight_estimation.body_parts import core_body_parts
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

rds = RDSAccessUtils()


query = """
    select * from prod.biomass_computations
    where pen_id=140
    and captured_at between '2020-12-01' and '2021-01-30'
    and akpd_score > 0.95;
"""
df = rds.extract_from_database(query)

mean_y_diffs = []
for idx, row in df.iterrows():
    count += 1
    ann = row.annotation
    ann_left_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    ann_right_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    y_diffs = []
    for bp in core_body_parts:
        y_diffs.append(ann_left_dict[bp][1] - ann_right_dict[bp][1])
    
    mean_y_diffs.append(np.mean(y_diffs))
    
df['mean_y_diff'] = mean_y_diffs



In [ ]:
df['crop_area'] = df.left_crop_metadata.apply(lambda x: x['width'] * x['height'])
df.index = pd.to_datetime(df.captured_at)
df['hour'] = df.index.hour
df['date'] = df.index.date.astype(str)

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(df.crop_area.resample('D', how=lambda x: x.mean()))
plt.axvline('2021-02-05', color='red', linestyle='--', label='camera swap')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Crop area (pixels squared)')
plt.title('Dale P3')
plt.grid()
plt.show()

In [ ]:
mean_y_diffs = []
for idx, row in df.iterrows():
    count += 1
    ann = row.annotation
    ann_left_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['leftCrop']}
    ann_right_dict = {item['keypointType']: [item['xFrame'], item['yFrame']] for item in ann['rightCrop']}
    y_diffs = []
    for bp in core_body_parts:
        y_diffs.append(ann_left_dict[bp][1] - ann_right_dict[bp][1])
    
    mean_y_diffs.append(np.mean(y_diffs))
    
df['mean_y_diff'] = mean_y_diffs



In [ ]:
df.index = pd.to_datetime(df.captured_at)

In [ ]:
plt.figure(figsize=(20, 10))
plt.plot(df.mean_y_diff.resample('D', how=lambda x: x.mean()))
plt.axvline('2020-12-29', color='red', linestyle='--', label='camera swap')
plt.legend()
plt.xlabel('Date')
plt.ylabel('Y-coordinate pixel deviation')
plt.grid()
plt.show()

In [ ]:
df.mean_y_diff.resample('D', how=lambda x: x.mean())